In [1]:
from itertools import chain, combinations, filterfalse
from tqdm import tqdm

In [72]:
from itertools import chain, combinations, filterfalse

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

def join_set(itemsets, k):
    return set(
        [itemset1.union(itemset2) for itemset1 in itemsets for itemset2 in itemsets if len(itemset1.union(itemset2)) == k]
    )

def itemsets_support(transactions, itemsets, min_support):
    support_count = {itemset: 0 for itemset in itemsets}
    for transaction in transactions:
        for itemset in itemsets:
            if itemset.issubset(transaction):
                support_count[itemset] += 1
    n_transactions = len(transactions)
    return {itemset: support / n_transactions for itemset, support in support_count.items() if support / n_transactions >= min_support}

def apriori(transactions, min_support):
    items = set(chain(*transactions))
    itemsets = [frozenset([item]) for item in items]
    itemsets_by_length = [set()]
    k = 1
    while itemsets:
        support_count = itemsets_support(transactions, itemsets, min_support)
        itemsets_by_length.append(set(support_count.keys()))
        k += 1
        itemsets = join_set(itemsets, k)
    frequent_itemsets = set(chain(*itemsets_by_length))
    return frequent_itemsets, itemsets_by_length

def association_rules(transactions, min_support, min_confidence):
    frequent_itemsets, itemsets_by_length = apriori(transactions, min_support)
    rules = []
    for itemset in frequent_itemsets:
        for subset in filterfalse(lambda x: not x, powerset(itemset)):
            antecedent = frozenset(subset)
            consequent = itemset - antecedent
            support_antecedent = len([t for t in transactions if antecedent.issubset(t)]) / len(transactions)
            # OLD: 
            support_itemset = len([t for t in transactions if itemset.issubset(t)]) / len(transactions)
            # NEW:
            # support_itemset = len([t for t in transactions if (antecedent | consequent).issubset(t)]) / len(transactions)
            # NEW:
            support_consequent = len([t for t in transactions if consequent.issubset(t)]) / len(transactions)
            lift = support_itemset / (support_antecedent * support_consequent)
            
            # OLD:
            # confidence = support_itemset / support_antecedent
            # NEW:
            confidence = support_itemset / support_antecedent if consequent else support_antecedent

            if confidence >= min_confidence and consequent:
                # OLD:
                # rules.append((antecedent, consequent, support_itemset, confidence))
                # NEW:
                rules.append((antecedent, consequent, support_itemset, confidence, lift))
    return rules

In [73]:
transactions = [
    {"beer", "wine", "cheese"},
    {"beer", "potato chips"},
    {"eggs", "flour", "butter", "cheese"},
    {"eggs", "flour", "butter", "beer", "potato chips"},
    {"wine", "cheese"},
    {"potato chips"},
    {"eggs", "flour", "butter", "wine", "cheese"},
    {"eggs", "flour", "butter", "beer", "potato chips"},
    {"wine", "beer"},
    {"beer", "potato chips"},
    {"butter", "eggs"},
    {"beer", "potato chips"},
    {"flour", "eggs"},
    {"beer", "potato chips"},
    {"eggs", "flour", "butter", "wine", "cheese"},
    {"beer", "wine", "potato chips", "cheese"},
    {"wine", "cheese"},
    {"beer", "potato chips"},
    {"wine", "cheese"},
    {"beer", "potato chips"}
]

In [74]:
min_support = 0.3
min_confidence = 0.5

rules = association_rules(transactions, min_support, min_confidence)
for antecedent, consequent, support, confidence, lift in rules:
    print(f"{set(antecedent)} => {set(consequent)} (support={support:.3f}, confidence={confidence:.3f}, lift={lift:.3f})")

{'potato chips'} => {'beer'} (support=0.450, confidence=0.900, lift=1.636)
{'beer'} => {'potato chips'} (support=0.450, confidence=0.818, lift=1.636)
{'butter'} => {'eggs'} (support=0.300, confidence=1.000, lift=2.857)
{'eggs'} => {'butter'} (support=0.300, confidence=0.857, lift=2.857)
{'wine'} => {'cheese'} (support=0.350, confidence=0.875, lift=2.187)
{'cheese'} => {'wine'} (support=0.350, confidence=0.875, lift=2.187)
{'flour'} => {'eggs'} (support=0.300, confidence=1.000, lift=2.857)
{'eggs'} => {'flour'} (support=0.300, confidence=0.857, lift=2.857)


In [71]:
min_support = 0.3
min_confidence = 0.5

rules = association_rules(transactions, min_support, min_confidence)
for antecedent, consequent, support, confidence, lift in rules:
    print(f"{set(antecedent)} => {set(consequent)} (support={support:.3f}, confidence={confidence:.3f}, lift={lift:.3f})")

{'potato chips'} => {'beer'} (support=0.450, confidence=0.900, lift=1.636)
{'beer'} => {'potato chips'} (support=0.450, confidence=0.818, lift=1.636)
{'butter'} => {'eggs'} (support=0.300, confidence=1.000, lift=2.857)
{'eggs'} => {'butter'} (support=0.300, confidence=0.857, lift=2.857)
{'wine'} => {'cheese'} (support=0.350, confidence=0.875, lift=2.187)
{'cheese'} => {'wine'} (support=0.350, confidence=0.875, lift=2.187)
{'flour'} => {'eggs'} (support=0.300, confidence=1.000, lift=2.857)
{'eggs'} => {'flour'} (support=0.300, confidence=0.857, lift=2.857)


In [38]:
transactions = [
    {"beer", "wine", "cheese"},
    {"beer", "potato chips"},
    {"eggs", "flour", "butter", "cheese"},
    {"eggs", "flour", "butter", "beer", "potato chips"},
    {"wine", "cheese"},
    {"potato chips"},
    {"eggs", "flour", "butter", "wine", "cheese"},
    {"eggs", "flour", "butter", "beer", "potato chips"},
    {"wine", "beer"},
    {"beer", "potato chips"},
    {"butter", "eggs"},
    {"beer", "potato chips"},
    {"flour", "eggs"},
    {"beer", "potato chips"},
    {"eggs", "flour", "butter", "wine", "cheese"},
    {"beer", "wine", "potato chips", "cheese"},
    {"wine", "cheese"},
    {"beer", "potato chips"},
    {"wine", "cheese"},
    {"beer", "potato chips"}
]

In [187]:
transactions = [
    {'1', '3', '4'},
    {'2', '3', '5'},
    {'1', '2', '3', '5'},
    {'2', '5'}
]

In [188]:
min_support = 0.5

In [189]:
# Apriori
items = set(chain(*transactions))  # Get unique items
itemsets = [frozenset([item]) for item in items]  # Get itemsets of length 1
itemsets_by_length = [set()]
itemsets

[frozenset({'3'}),
 frozenset({'1'}),
 frozenset({'2'}),
 frozenset({'4'}),
 frozenset({'5'})]

In [190]:
k = 1
support_count = itemsets_support(transactions, itemsets, min_support)
itemsets_by_length.append(set(support_count.keys()))
k += 1
support_count, itemsets_by_length, bool(support_count), bool(itemsets)

({frozenset({'3'}): 0.75,
  frozenset({'1'}): 0.5,
  frozenset({'2'}): 0.75,
  frozenset({'5'}): 0.75},
 [set(),
  {frozenset({'3'}), frozenset({'1'}), frozenset({'2'}), frozenset({'5'})}],
 True,
 True)

In [191]:
itemsets = join_set(support_count.keys(), k)
itemsets

{frozenset({'1', '3'}),
 frozenset({'1', '5'}),
 frozenset({'2', '3'}),
 frozenset({'2', '5'}),
 frozenset({'3', '5'}),
 frozenset({'1', '2'})}

In [192]:
k = 2
support_count = itemsets_support(transactions, itemsets, min_support)
itemsets_by_length.append(set(support_count.keys()))
k += 1
itemsets = join_set(support_count.keys(), k)
support_count, itemsets_by_length, itemsets, bool(support_count), bool(itemsets)

({frozenset({'1', '3'}): 0.5,
  frozenset({'2', '3'}): 0.5,
  frozenset({'2', '5'}): 0.75,
  frozenset({'3', '5'}): 0.5},
 [set(),
  {frozenset({'3'}), frozenset({'1'}), frozenset({'2'}), frozenset({'5'})},
  {frozenset({'2', '3'}),
   frozenset({'2', '5'}),
   frozenset({'1', '3'}),
   frozenset({'3', '5'})}],
 {frozenset({'1', '3', '5'}),
  frozenset({'2', '3', '5'}),
  frozenset({'1', '2', '3'})},
 True,
 True)

In [193]:
k = 3
support_count = itemsets_support(transactions, itemsets, min_support)
itemsets_by_length.append(set(support_count.keys()))
k += 1
itemsets = join_set(support_count.keys(), k)
support_count, itemsets_by_length, itemsets, bool(support_count), bool(itemsets)

({frozenset({'2', '3', '5'}): 0.5},
 [set(),
  {frozenset({'3'}), frozenset({'1'}), frozenset({'2'}), frozenset({'5'})},
  {frozenset({'2', '3'}),
   frozenset({'2', '5'}),
   frozenset({'1', '3'}),
   frozenset({'3', '5'})},
  {frozenset({'2', '3', '5'})}],
 set(),
 True,
 False)

In [194]:
frequent_itemsets = set(chain(*itemsets_by_length))
frequent_itemsets

{frozenset({'2'}),
 frozenset({'1', '3'}),
 frozenset({'2', '3'}),
 frozenset({'1'}),
 frozenset({'2', '5'}),
 frozenset({'3'}),
 frozenset({'5'}),
 frozenset({'3', '5'}),
 frozenset({'2', '3', '5'})}

In [195]:
# association_rules
rules = []
itemset = list(frequent_itemsets)[0]
# subset = list(filterfalse(lambda x: not x, powerset(itemset)))[0]
subset = list(frequent_itemsets)[1]
itemset, subset

(frozenset({'2'}), frozenset({'1', '3'}))